In [58]:
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit,transpile
from qiskit_aer import Aer, AerSimulator
import Our_Qiskit_Functions as oq


import numpy as np
import math as m
S_simulator = Aer.backends('statevector_simulator')[0]
M_simulator = Aer.backends('qasm_simulator')[0]

In [ ]:
q=QuantumRegister(3,name='q')
anc=QuantumRegister(1,name='anc')
qc=QuantumCircuit(q,anc,name='qc')
qc.h(q[0])
qc.h(q[1])
qc.h(q[2])
qc.x(anc[0])
oq.Wavefunction(qc,systems=[3,1],show_system=[True,False])

qc.h(anc[0])
f=oq.Blackbox_g_DJ(3,qc,q,anc)
if f[0]=="constant":
    A=1
else:
    A=2

qc.h(anc[0]) 
oq.Wavefunction(qc,systems=[3,A],show_system=[True,False])
print("ftype is ",f[0])
if len(f)>1:
    print("states mapped to 1 ",f[1:len(f)])
"""The code above performs the Deutsch-Jozsa Algorithm up to the step where we apply our blackbox g. 
Run the cell of code a couple times until you come across a case where f is balanced. 
When you do, you should notice that exactly half of the states in the system pick up a negative sign. 
And, these states exactly match up with the states that get mapped to 1 by the embedded f, printed at the bottom."""
    

0.35355 |000>|1>    0.35355 |100>|1>    0.35355 |010>|1>    0.35355 |110>|1>    0.35355 |001>|1>    0.35355 |101>|1>    0.35355 |011>|1>    0.35355 |111>|1>    
-0.35355 |000>|10>    -0.35355 |100>|10>    0.35355 |010>|10>    0.35355 |110>|10>    -0.35355 |001>|10>    -0.35355 |101>|10>    0.35355 |011>|10>    0.35355 |111>|10>    
ftype is  balanced
states mapped to 1  ['|100>', '|101>', '|000>', '|001>']


In [49]:
#our algorithm calls for another Hadamard Transformation of our system, followed by a measurement:
q=QuantumRegister(3,name='q')
anc=QuantumRegister(1,name='anc')
c=ClassicalRegister(3,name='c')
qc=QuantumCircuit(q,anc,c,name='qc')
qc.h(q[0])
qc.h(q[1])
qc.h(q[2])
qc.x(anc[0])
oq.Wavefunction(qc,systems=[3,1],show_system=[True,False])

qc.h(anc[0])
f=oq.Blackbox_g_DJ(3,qc,q,anc)
if f[0]=="constant":
    A=1
else:
    A=2

qc.h(anc[0]) 
oq.Wavefunction(qc,systems=[3,A],show_system=[True,False])
qc.h(q[0])
qc.h(q[1])
qc.h(q[2])
oq.Wavefunction(qc,systems=[3,A],show_system=[True,False])
qc.measure(q,c)


# Use the qasm simulator
M_simulator = Aer.get_backend('qasm_simulator')

# Execute the circuit with 1 shot
job = M_simulator.run(qc, shots=1)

# Get the results
result = job.result()
counts = result.get_counts(qc)

print("Measurement result:", counts)
#print whether system is balanced or not
if list(counts.keys())[0]=="000":
    print("f is constant")
else:
    print("f is balanced")

0.35355 |000>|1>    0.35355 |100>|1>    0.35355 |010>|1>    0.35355 |110>|1>    0.35355 |001>|1>    0.35355 |101>|1>    0.35355 |011>|1>    0.35355 |111>|1>    
0.35355 |000>|1>    0.35355 |100>|1>    0.35355 |010>|1>    0.35355 |110>|1>    0.35355 |001>|1>    0.35355 |101>|1>    0.35355 |011>|1>    0.35355 |111>|1>    
1.0 |000>|1>    
Measurement result: {'000': 1}
f is constant


In [56]:
#deutsch-jozsa example with 2 qubits
import numpy as np
from qiskit import QuantumRegister, QuantumCircuit
from qiskit.visualization import plot_bloch_multivector
from qiskit.quantum_info import Statevector as Wavefunction

# Create quantum and ancilla registers
q = QuantumRegister(2, name='q')      # main 2 qubits
anc = QuantumRegister(1, name='anc')  # ancilla qubit

# Two quantum circuits for comparison
con1_qc = QuantumCircuit(q, anc, name='qc1')
con2_qc = QuantumCircuit(q, anc, name='qc2')

# ---- Step 1: Apply H to all qubits ----
for i in np.arange(2):
    con1_qc.h(q[int(i)])
    con2_qc.h(q[int(i)])


# Apply H to ancilla and flip it to |1> via XH
con1_qc.h(anc[0])
con1_qc.x(anc[0])
con2_qc.x(anc[0])
con2_qc.h(anc[0])

print("___ Before g ___")
print(Wavefunction(con1_qc))
print(Wavefunction(con2_qc))

# ---- Step 2: Define oracle (balanced) ----

con2_qc.x(q[0])
con2_qc.x(q[1])
con2_qc.x(anc[0])

print("\n___ After g (f type: balanced) ___")
print(Wavefunction(con1_qc))
print(Wavefunction(con2_qc))

# ---- Step 3: Apply final Hadamard transforms ----
for i in np.arange(2):
    con1_qc.h(q[int(i)])
    con2_qc.h(q[int(i)])
con1_qc.h(anc[0])
con2_qc.h(anc[0])

print("\n___ After H^3 ___")
oq.Wavefunction(con1_qc)
oq.Wavefunction(con2_qc)

print(con1_qc.draw())
print(con2_qc.draw())

___ Before g ___
Statevector([0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j,
             0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j,
             0.35355339+0.j, 0.35355339+0.j],
            dims=(2, 2, 2))
Statevector([ 0.35355339+0.j,  0.35355339+0.j,  0.35355339+0.j,
              0.35355339+0.j, -0.35355339+0.j, -0.35355339+0.j,
             -0.35355339+0.j, -0.35355339+0.j],
            dims=(2, 2, 2))

___ After g (f type: balanced) ___
Statevector([0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j,
             0.35355339+0.j, 0.35355339+0.j, 0.35355339+0.j,
             0.35355339+0.j, 0.35355339+0.j],
            dims=(2, 2, 2))
Statevector([-0.35355339+0.j, -0.35355339+0.j, -0.35355339+0.j,
             -0.35355339+0.j,  0.35355339+0.j,  0.35355339+0.j,
              0.35355339+0.j,  0.35355339+0.j],
            dims=(2, 2, 2))

___ After H^3 ___
1.0 |000>    
-1.0 |001>    
     ┌───┐┌───┐     
q_0: ┤ H ├┤ H ├─────
     ├───┤├───┤     
q_1: ┤ H ├┤ H ├─────
     ├───┤├───┤

In [67]:
from qiskit import QuantumRegister, QuantumCircuit
  # custom library for visualization / oracle definition

# --- STEP 1: Define Quantum Registers ---
q = QuantumRegister(3, name='q')      # 3 input qubits
anc = QuantumRegister(1, name='anc')  # 1 ancilla qubit (output qubit)
DJ_qc = QuantumCircuit(q, anc, name='qc')  # Deutsch–Jozsa circuit

# --- STEP 2: Initialize all qubits in superposition ---
for i in range(3):
    DJ_qc.h(q[i])     # Apply H to each input qubit
DJ_qc.h(anc[0])       # Apply H to ancilla
DJ_qc.x(anc[0])       # Flip ancilla to |1⟩

print('___ Before g ___')
oq.Wavefunction(DJ_qc, systems=[3, 1], show_systems=[True, False])

# --- STEP 3: Apply Oracle g ---
DJ_qc.h(anc[0])    # Prepare ancilla in |-⟩ state
f = oq.Blackbox_g_DJ(3, DJ_qc, q, anc)  # Apply oracle (hidden function f)

# Determine if f is constant or balanced
if f[0] == 'constant':
    A = 1
else:
    A = 2

# --- STEP 4: Show State After Oracle ---
DJ_qc.h(anc[0])
print(' ')
print('___ After g ___')
oq.Wavefunction(DJ_qc, systems=[3, A], show_systems=[True, False])

# --- STEP 5: Apply Final Hadamard Transforms ---
for i in range(3):
    DJ_qc.h(q[i])

print('\n___ After H^3 ___')
oq.Wavefunction(DJ_qc, systems=[3, A], show_systems=[True, False])

# --- STEP 6: Print Result ---
print(' ')
print('f type:', f[0])
if len(f) > 1:
    print('- States mapped to 1:', f[1:len(f)])
    print('- Note that the state |000⟩ is not in our final system!')


___ Before g ___
0.25 |000>    0.25 |100>    0.25 |010>    0.25 |110>    0.25 |001>    0.25 |101>    0.25 |011>    0.25 |111>    0.25 |000>    0.25 |100>    0.25 |010>    0.25 |110>    0.25 |001>    0.25 |101>    0.25 |011>    0.25 |111>    
 
___ After g ___
0.25 |000>    0.25 |100>    0.25 |010>    0.25 |110>    0.25 |001>    0.25 |101>    0.25 |011>    0.25 |111>    -0.25 |000>    -0.25 |100>    0.25 |010>    -0.25 |110>    0.25 |001>    0.25 |101>    -0.25 |011>    0.25 |111>    

___ After H^3 ___
0.70711 |000>    -0.35355 |001>    0.35355 |101>    -0.35355 |011>    -0.35355 |111>    
 
f type: balanced
- States mapped to 1: ['|011>', '|000>', '|110>', '|100>']
- Note that the state |000⟩ is not in our final system!


In [ ]:
#example 010 as control gate

from qiskit import QuantumRegister, QuantumCircuit
 # Assuming 'oq' is your custom module for visualization and gates

# Define quantum registers
q = QuantumRegister(3, name='q')
trgt = QuantumRegister(1, name='trgt')
anc = QuantumRegister(1, name='anc')

# Define quantum circuit
qc_010 = QuantumCircuit(q, trgt, anc, name='qc')

# --- Initial State Preparation ---
qc_010.id(q[0])
qc_010.h(q[1])
qc_010.id(q[2])
qc_010.x(trgt[0])
qc_010.h(trgt[0])

print('___ Initial State ___')
oq.Wavefunction(qc_010, systems=[3, 1, 1], show_systems=[True, True, False])

# --- Apply n_NOT operation ---
qc_010.x(q[0])
qc_010.x(q[2])
oq.n_NOT(qc_010, q, trgt[0], anc)
qc_010.x(q[0])
qc_010.x(q[2])

print()
print('___ After n_NOT ___')
oq.Wavefunction(qc_010, systems=[3, 1, 1], show_systems=[True, True, False])
#These X gates transform our desired control state into the state of all 1’s, s
# uch that the n NOT operation will work, and then back to the original state:

___ Initial State ___
0.5 |000>|0>    0.5 |010>|0>    -0.5 |000>|1>    -0.5 |010>|1>    

___ After n_NOT ___
0.5 |000>|0>    -0.5 |010>|0>    -0.5 |000>|1>    0.5 |010>|1>    
